In [1]:
import cv2 
import numpy as np

LABELS = ['pineapple','apple']  # 이미지 라벨링

# 파인애플과 사과에 대한 확률이 30% 만 넘으면  그 사물로 보겠다.
CONFIDENCE = 0.3

THRESHOLD = 0.3 # NMS(Num Max Suppression)

net = cv2.dnn.readNetFromDarknet('C:/Data/image/yolov4_custom.cfg','C:/Data/image/yolov4_custom_last.weights') # 환경 구성 파일, 가중치

In [22]:
img_path = "C:/Data/image/Pineapple_apple.jpeg"

In [16]:
# 사물검출 함수
def main(img_path):
    img = cv2.imread(img_path) 
    H, W, _ = img.shape 
   
   # scalefactor : 정규화
   # swapRB=True : BGR -> RGB 
    blob = cv2.dnn.blobFromImage(img, scalefactor=1/255., size=(416, 416), swapRB=True) 

    net.setInput(blob) # setInput = predict
    output = net.forward() # output
 
    boxes, confidences, class_ids = [], [], []  
 
    for det in output: # output[:4] : x, y, w, h / output[5:] : score
        box = det[:4] 
        scores = det[5:] # label rates -> [pineapple rate, apple rate]
        class_id = np.argmax(scores) # [0.2, 0.8] ->  index 1
        confidence = scores[class_id] # confidence = 0.8 index
   
        if confidence > CONFIDENCE:  
            cx, cy, w, h = box * np.array([W, H, W, H]) 
            x = cx - (w / 2) 
            y = cy - (h / 2) # 박스를 바운딩할 x 와 y 좌표

            boxes.append([int(x), int(y), int(w), int(h)])  # 박스 바운딩할 위치 정보
            confidences.append(float(confidence))  # 확률 정보
            class_ids.append(class_id) 
   
    # Num Max Suppression
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE, THRESHOLD)
   
    if len(idxs) > 0: # object detection True
        for i in idxs.flatten(): # 박싱
            x, y, w, h = boxes[i]  
            
            cv2.rectangle(img, pt1=(x, y), pt2=(x + w, y + h), color=(0, 0, 255), thickness=2) # red border
            # object detection 정보
            cv2.putText(img, text='%s %.2f %d' % (LABELS[class_ids[i]], confidences[i], w), org=(x, y - 10),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.5, color=(0, 0, 255), thickness=2)
    
    # 이미지 저장
    cv2.imwrite('C:/Data/image/pineapple_apple_detection.png', img)

In [23]:
main(img_path)